In [42]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
import statsmodels.api as sm
from bokeh.models import Legend

output_notebook()


Loading BokehJS ...

In [43]:
missing = pd.read_csv("Wholesale customers data-missing.csv")
whole = pd.read_csv("Wholesale customers data.csv")


In [44]:
missing.describe()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
count,440.000000,440.000000,440.000000,439.000000,438.000000,440.000000,438.000000,439.000000
mean,1.322727,2.543182,12000.297727,5795.719818,7972.655251,3071.931818,2891.719178,1525.717540
std,0.468052,0.774272,12647.328865,7388.788560,9517.027646,4854.673333,4776.332439,2823.267359
min,1.000000,1.000000,3.000000,55.000000,137.000000,25.000000,3.000000,3.000000
25%,1.000000,2.000000,3127.750000,1532.000000,2155.500000,742.250000,256.250000,407.500000
50%,1.000000,3.000000,8504.000000,3620.000000,4755.500000,1526.000000,820.500000,964.000000
75%,2.000000,3.000000,16933.750000,7196.500000,10675.250000,3554.250000,3948.000000,1821.500000
max,2.000000,3.000000,112151.000000,73498.000000,92780.000000,60869.000000,40827.000000,47943.000000


In [45]:
##data contain missing value
missing[missing.isnull().any(axis=1)]

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
75,1,3,20398,1137.0,NaN,4407,3.0,975.0
172,1,3,955,5479.0,NaN,333,2840.0,707.0
180,1,3,12356,NaN,8887.0,402,1382.0,2794.0
225,1,1,12680,3243.0,4157.0,660,NaN,786.0
274,1,3,894,1703.0,1841.0,744,759.0,NaN
370,2,3,39679,3944.0,4955.0,1364,NaN,2235.0


In [46]:
### missing1 is data dropping NA's
missing1 = missing.dropna(axis=0,how="any")
#print(missing1.shape)
###correlation matrix
missing1.corr()

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
Channel,1.000000,0.061382,-0.180263,0.461940,0.610884,-0.205114,0.638856,0.054816
Region,0.061382,1.000000,0.054787,0.032920,0.008443,-0.021214,-0.000989,0.044222
Fresh,-0.180263,0.054787,1.000000,0.102470,-0.010642,0.348460,-0.100296,0.244921
Milk,0.461940,0.032920,0.102470,1.000000,0.727700,0.123466,0.661271,0.406431
Grocery,0.610884,0.008443,-0.010642,0.727700,1.000000,-0.041297,0.924786,0.205115
Frozen,-0.205114,-0.021214,0.348460,0.123466,-0.041297,1.000000,-0.133332,0.391815
Detergents_Paper,0.638856,-0.000989,-0.100296,0.661271,0.924786,-0.133332,1.000000,0.069348
Delicassen,0.054816,0.044222,0.244921,0.406431,0.205115,0.391815,0.069348,1.000000


In [47]:
###Method 1: Use column mean for missing data
missing2 = missing.copy()
missing2.at[[75,172],"Grocery"] = missing2["Grocery"].mean()
missing2.at[[225,370],"Detergents_Paper"] = missing2["Detergents_Paper"].mean()
missing2.at[180,"Milk"] = missing2["Milk"].mean()
missing2.at[274,"Delicassen"] = missing2["Delicassen"].mean()

In [48]:
###Method 2: Use Bining method
frozen = missing1["Frozen"]
missing1.groupby(pd.cut(frozen, bins=100)).mean().head(10) 
## the reason why I use large number of bins = 100 is that data frozen has very larger outliers

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
Frozen,,,,,,,,
"(-35.844, 633.44]",1.410526,2.652632,7291.178947,5315.747368,9507.631579,311.042105,3852.105263,910.894737
"(633.44, 1241.88]",1.430108,2.548387,9343.494624,6689.279570,8955.107527,929.451613,3355.612903,1221.505376
"(1241.88, 1850.32]",1.444444,2.462963,9494.518519,5576.555556,8116.592593,1544.018519,3204.296296,1478.074074
"(1850.32, 2458.76]",1.200000,2.514286,11533.028571,4093.285714,5257.942857,2193.800000,1767.228571,1282.971429
"(2458.76, 3067.2]",1.387097,2.612903,14468.870968,5547.451613,7564.806452,2737.548387,2883.096774,1048.645161
"(3067.2, 3675.64]",1.250000,2.600000,13894.100000,7147.850000,10319.850000,3356.000000,4243.200000,1593.500000
"(3675.64, 4284.08]",1.214286,2.357143,11588.214286,2916.857143,4810.785714,3925.500000,1653.928571,1424.642857
"(4284.08, 4892.52]",1.214286,2.357143,16997.071429,7012.785714,8008.000000,4543.285714,3013.357143,1778.357143
"(4892.52, 5500.96]",1.111111,2.000000,18573.222222,6568.333333,7412.777778,5244.333333,1598.777778,4784.888889


In [49]:
missing3 = missing.copy()
###fill in missing values with mean got from Binning method.
missing3.at[75,"Grocery"] = 8008
missing3.at[172,"Grocery"] = 9507.63
missing3.at[180,"Milk"] = 5315.75
missing3.at[225,"Detergents_Paper"] = 3355.61
missing3.at[274,"Delicassen"] = 1221.50
missing3.at[370,"Detergents_Paper"] = 3204.29

In [50]:
###Method 3: Using regression; Milk, Grocery, Detergents_Paper are highly correlated, 

model1 = sm.OLS(missing1["Grocery"], missing1["Detergents_Paper"]).fit() 
predictions_grocery = model1.predict([3,2840])

model2 = sm.OLS(missing1["Detergents_Paper"],missing1["Grocery"]).fit()
predictions_detergantspaper = model2.predict([4157,4955])

model3 = sm.OLS(missing1["Milk"],missing1["Grocery"]).fit()
predictions_milk = model3.predict([8887])


model4 = sm.OLS(missing1["Delicassen"], missing1["Milk"]).fit()
predictions_delicassen = model4.predict([1703])

###fill in missing values with predictions using linear regression
missing4 = missing.copy()
missing4.at[[75,172],"Grocery"] = predictions_grocery
missing4.at[[225,370],"Detergents_Paper"] = predictions_detergantspaper
missing4.at[180,"Milk"] = predictions_milk
missing4.at[274,"Delicassen"] = predictions_delicassen



In [52]:
###Plot filling missing values and whole data points

source_whole = ColumnDataSource(data=dict(np.log(whole)))
source_method1 = ColumnDataSource(data=dict(np.log(missing2)))
source_method2 = ColumnDataSource(data=dict(np.log(missing3)))
source_method3 = ColumnDataSource(data=dict(np.log(missing4)))

TOOLS = "box_select,lasso_select,help"

p1 = figure(tools=TOOLS,width=300, height=300, x_axis_label ="Grocery", y_axis_label = "Milk",title="Grocery_Milk")
p11 = p1.circle("Grocery","Milk",source=source_whole, alpha=0.5)
p12 = p1.circle(np.log([3, 6536, 8887]),np.log([1137,5479,6036]),fill_color="Blue",fill_alpha=1,size=6)
p13 = p1.diamond(np.log(np.concatenate((np.repeat(missing2["Grocery"].mean(),2),[8887]),axis=0)),
           np.log(np.concatenate(([1137,5479], [missing2["Milk"].mean()]),axis=0)),fill_color="Red",fill_alpha=1,size=6)

p14 = p1.square(np.log([8008,9507.63, 8887]),np.log([1137,5479,5315.75]),fill_color="navy",fill_alpha=1,size=6)
p15 = p1.triangle(np.log(np.concatenate((predictions_grocery,[8887]),axis=0)),np.log(np.concatenate(([1137,5479], predictions_milk),axis=0)),
            fill_color="Yellow",fill_alpha=1,size=6)



p2 = figure(tools=TOOLS,width=300, height=300, x_axis_label ="Grocery", y_axis_label = "Detergents_Paper",title="Grocery_Detergents_Paper")
p21 = p2.circle("Grocery","Detergents_Paper", source=source_whole, alpha=0.5)
p22 = p2.circle(np.log([4157,4955]),np.log([761,523]),fill_color="Blue",fill_alpha=1,size=10)

p23 = p2.diamond(np.log([4157,4955]),np.log(np.repeat(missing2["Detergents_Paper"].mean(),2)),fill_color="Red",fill_alpha=1,size=10)

p24 = p2.square(np.log([4157,4955]),np.log([3355.61,3204.29]),fill_color="navy",fill_alpha=1,size=10)
p25 = p2.triangle(np.log([4157,4955]),np.log(predictions_detergantspaper),fill_color="Yellow",fill_alpha=1,size=10)




p3 = figure(tools=TOOLS,width=300, height=300, x_axis_label ="Grocery", y_axis_label = "Delicassen",title="Grocery_Delicassen")
p31 = p3.circle("Grocery","Delicassen", alpha=0.5,source=source_whole)

p32 = p3.circle(np.log(1841),np.log(1153),fill_color="Blue",fill_alpha=1,size=10)

p33 = p3.diamond(np.log(1841),np.log(missing2["Delicassen"].mean()),fill_color="Red",fill_alpha=1,size=10)

p34 = p3.square(np.log(1841),np.log(1221.50),fill_color="navy",fill_alpha=1,size=10)
p35 = p3.triangle(np.log(1841),np.log(predictions_delicassen),fill_color="Yellow",fill_alpha=1,size=10)


legend1 = Legend(items=[("True Value", [p12]),("Method I: Column Mean" , [p13]),
                       ("Method II: Bining" , [p14]),("Method III: Regression" , [p15])], location=(10, 12))

p1.add_layout(legend1, 'below')

legend2 = Legend(items=[("True Value", [p22]),("Method I: Column Mean" , [p23]),
                       ("Method II: Bining" , [p24]),("Method III: Regression" , [p25])], location=(10, 12))



p2.add_layout(legend2, 'below')

legend3 = Legend(items=[("True Value", [p32]),("Method I: Column Mean" , [p33]),
                       ("Method II: Bining" , [p34]),("Method III: Regression" , [p35])], location=(10, 12))



p3.add_layout(legend3, 'below')



p = gridplot([[p1, p2, p3]])



show(p)